In [1]:
from src.data import DataHandler
from src.engine import BacktestEngine
from src.portofolio import Portfolio, PercentageFeeModel, LinearSlippageModel, RebalanceMode
from src.reporting import ReportGenerator

from src.strategies import PairsTradingStrategy, SMACrossStrategy
from src.benchmark import Benchmark

### Pairs trading (National Grid / United Utilities)

In [13]:
tickers = ["NG.L", "UU.L"]
START_DATE = '2005-01-01'
END_DATE = '2025-07-31'
INITIAL_CAPITAL = 100_000

handler = DataHandler(tickers=tickers, start_date=START_DATE, end_date=END_DATE)
data = handler.download()

strategy = PairsTradingStrategy(tickers=tickers,
                                window=60,
                                entry_z=2.0,
                                exit_z=0.0)

portfolio = Portfolio(initial_capital=INITIAL_CAPITAL,
                        fee_model=PercentageFeeModel(fee_rate=0.001), #0.1%
                        slippage_model=LinearSlippageModel(base_bps=5, impact_per_share=0.0001),
                        rebalance_mode=RebalanceMode.CONTINUOUS_WITH_BUFFER,
                        trade_buffer=0.05, #5%
                        max_allocation_pct=0.2) #20% per trade

engine = BacktestEngine()
metrics = engine.run(data, strategy, portfolio, liquidate_at_end=True) 

bench_config = {tickers[0]: 0.5, tickers[1]: 0.5}
bench_engine = Benchmark(data, composition=bench_config)
bench_series = bench_engine.calculate(initial_capital=INITIAL_CAPITAL)

reporter = ReportGenerator(metrics=metrics,
                           benchmark_equity=bench_series,
                           strategy_name=f"Pairs Trading {tickers[0]} / {tickers[1]}")
reporter.generate("Pairs_Trading_Report.html")

Generating the Dashboard for Pairs Trading NG.L / UU.L...
Dashboard saved : /Users/carlm/Documents/Dauphine/Python projects/Backtest_engine/reports/Pairs_Trading_Report.html


### SMA Crossover (Safran)

In [11]:
ticker = ["SAF.PA"]
START_DATE = '2022-01-01'
END_DATE = '2025-07-31'
SHORT_WINDOW, LONG_WINDOW = 20, 60
INITIAL_CAPITAL = 100_000

handler = DataHandler(tickers=ticker, start_date=START_DATE, end_date=END_DATE)
data = handler.download()

strategy = SMACrossStrategy(ticker=ticker, short_window=SHORT_WINDOW, long_window=LONG_WINDOW)

portfolio = Portfolio(initial_capital=INITIAL_CAPITAL,
                        fee_model=PercentageFeeModel(fee_rate=0.001), #0.1%
                        slippage_model=LinearSlippageModel(base_bps=5, impact_per_share=0.0001),
                        rebalance_mode=RebalanceMode.SIGNAL_CHANGE,
                        max_allocation_pct=0.2) #20% per trade

engine = BacktestEngine()
metrics = engine.run(data, strategy, portfolio, liquidate_at_end=True)

bench_config = {ticker[0]: 1.0}
bench_engine = Benchmark(data, composition=bench_config)
bench_series = bench_engine.calculate(initial_capital=INITIAL_CAPITAL)

reporter = ReportGenerator(metrics=metrics,
                           benchmark_equity=bench_series,
                           strategy_name=f"{ticker} SMA({SHORT_WINDOW}/{LONG_WINDOW}) Crossover Strategy")
reporter.generate("SMA_Crossover_Report.html")


Closing of all positions...
Generating the Dashboard for ['SAF.PA'] SMA(20/60) Crossover Strategy...
Dashboard saved : /Users/carlm/Documents/Dauphine/Python projects/Backtest_engine/reports/SMA_Crossover_Report.html
